In [1]:
import numpy as np
import slideio
import cv2
import json
import matplotlib.path as mpltPath
import os
import warnings
import tifffile
from skimage import exposure
from skimage.exposure import match_histograms
import xml.etree.ElementTree as ET
import pandas as pd
from PIL import Image
from math import sqrt
import matplotlib.pyplot as plt
from glob import glob
from copy import copy
from tqdm.notebook import tqdm
import warnings
import staintools
import shutil
from sklearn.model_selection import train_test_split

In [2]:
folder_list=  glob('../../data/gcu/tile/**/')
label_df=pd.read_csv('../../data/gcu/P53&MMR.csv')
label_df

,PathologyNumber,RecriptDate,P53,MLH1,MSH2,MSH6,PMS2,MMR status,"Subtype(1,endometrioid;2,serous;3,clear;4mixed;5etc;6MMMT)"
0,S10-01345,2010-01-18,Wild,0,1,1,0,Defect,1
1,S10-01350,2010-01-18,Wild,1,1,1,1,Normal,1
2,S10-03151,2010-02-08,Wild,1,1,1,1,Normal,1
3,S10-06438,2010-03-19,Wild,1,1,1,1,Normal,1
4,S10-07753,2010-04-05,Wild,1,1,1,1,Normal,1
...,...,...,...,...,...,...,...,...,...
320,S22-19730,2022-08-23,Wild,0,1,1,0,Defect,1
321,S22-19946,2022-08-25,Wild,1,1,1,1,Normal,1
322,S22-20342,2022-08-30,Wild,1,1,1,1,Normal,1
323,S22-21614,2022-09-15,Wild,1,1,1,1,Normal,1


In [3]:
y=[]
x=[]
y1=[]
for i in tqdm(range(len(folder_list))):
    PathologyNumber=os.path.basename(os.path.dirname(folder_list[i])).split('-')[0]+'-'+os.path.basename(os.path.dirname(folder_list[i])).split('-')[1]
    if len(label_df.loc[label_df['PathologyNumber']==PathologyNumber])!=0:
        x.append(folder_list[i])
        mmrd=label_df.loc[label_df['PathologyNumber']==PathologyNumber]['MMR status']
        p53=label_df.loc[label_df['PathologyNumber']==PathologyNumber]['P53']
        if mmrd.reset_index()['MMR status'][0]=='Defect':
            y.append(1)
        else:
            y.append(0)
            
        if p53.reset_index()['P53'][0]=='Wild':
            y1.append(0)
        else:
            y1.append(1)
    

  0%|          | 0/1110 [00:00<?, ?it/s]

In [4]:
X_train, X_test, MMR_train, MMR_test= train_test_split(x,y, test_size=0.1,stratify=y, random_state=123)
X_train1, X_test1, P53_train, P53_test= train_test_split(x,y1, test_size=0.1,stratify=y, random_state=123)
os.path.basename(os.path.dirname(folder_list[i]))
for i in tqdm(range(len(X_train))):
    if MMR_train[i]==1:
        shutil.copytree(X_train[i],'../../data/dataset/gcu/MMRd/train/Defect/'+os.path.basename(os.path.dirname(X_train[i])))
    else:
        shutil.copytree(X_train[i],'../../data/dataset/gcu/MMRd/train/Normal/'+os.path.basename(os.path.dirname(X_train[i])))

for i in tqdm(range(len(X_test))):
    if MMR_train[i]==1:
        shutil.copytree(X_test[i],'../../data/dataset/gcu/MMRd/test/Defect/'+os.path.basename(os.path.dirname(X_test[i])))
    else:
        shutil.copytree(X_test[i],'../../data/dataset/gcu/MMRd/test/Normal/'+os.path.basename(os.path.dirname(X_test[i])))
    
for i in tqdm(range(len(X_train1))):
    if P53_train[i]==1:
        shutil.copytree(X_train1[i],'../../data/dataset/gcu/P53/train/Missense/'+os.path.basename(os.path.dirname(X_train1[i])))
    else:
        shutil.copytree(X_train1[i],'../../data/dataset/gcu/P53/train/Wild/'+os.path.basename(os.path.dirname(X_train1[i])))

for i in tqdm(range(len(X_test1))):
    if P53_test[i]==1:
        shutil.copytree(X_test1[i],'../../data/dataset/gcu/P53/test/Missense/'+os.path.basename(os.path.dirname(X_test1[i])))
    else:
        shutil.copytree(X_test1[i],'../../data/dataset/gcu/P53/test/Wild/'+os.path.basename(os.path.dirname(X_test1[i])))

  0%|          | 0/817 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

  0%|          | 0/817 [00:00<?, ?it/s]

  0%|          | 0/91 [00:00<?, ?it/s]

In [93]:
MMR_train[i]

0

In [91]:
'../../data/dataset/gcu/MMRd/train/Normal/'+os.path.basename(X_train[i])

''